# Newsvendor problem (1-d)


### Parameters

- $h=b=1$
- $F$ Underlying distribution
    - $F_1 \sim N(50, 50)$
    - $F_2 \sim \exp(1/50)$
- $\mathbf N, |\mathbf N| = N$ is # of realizations

#### type of distribution set:

- `likelihood`, see `Wang, Zizhuo, Peter W Glynn, and Yinyu Ye. 2016. “Likelihood Robust Optimization for Data-Driven Problems.” Computational Management Science 13 (2): 241–61. https://doi.org/10.1007/s10287-015-0240-3.`

### The Models

#### Newsvendor primal

$$\begin{aligned} 
 \textsf{ minimize the worse-case expected cost: }  \\
 & \min_x \max_p \mathbb E_p(\mathbf h) \\
 \textsf{ loss function: }  \\
 &  \mathbf h, \quad h_i = b(d_i - x)^+ + h(x-d_i)^+\\
 \textbf {s.t. }  & \\
 & \mathbf p \in \mathcal D_d, \quad \textsf{ where } \mathcal D_d \textsf{ is some valid distribution set }
\end{aligned}$$

#### Scarf (DRO)

$$x_{\text {scar } f}^{*}=\hat{\mu}+\frac{\hat{\sigma}}{2}(\sqrt{\frac{b}{h}}-\sqrt{\frac{h}{b}})$$

we discuss the distribution set:


#### LRO: log-likelihood

where

$$\begin{aligned} 
 \textsf{likehood: } & \\
 & \sum_{i=0}^{n} N_{i} \log p_{i} \geq \gamma \\ 
 & \sum_{i=0}^{n} p_{i}=1, \quad p_{i} \geq 0, \forall i
 \end{aligned}$$

we have the following:

$$\begin{aligned}
  & \max_{x, ...} 
  \theta + \beta \gamma + \beta N + t \\
  \textbf {s.t.} \\
  & (\mathbf q, \beta\mathbf N, t) \in \mathcal K_{\exp} \\
  & \beta \ge 0 \\
  & \mathbf q \equiv - \mathbf h - \theta \mathbf 1 \ge 0\\
  & \mathbf q + \theta \mathbf 1 +  b\cdot(d - x) \le 0\\
  & \mathbf q +  \theta \mathbf 1 + h\cdot(x - d) \le 0\\
  & x \in D 
\end{aligned}$$

using estimator:

$$\gamma^{*}=\sum_{i=1}^{n} N_{i} \log \frac{N_{i}}{N}-\frac{1}{2} \chi_{n-1,1-\alpha}^{2}$$

worst-case probability:

$$p^\star = \frac{\beta \mathbf N}{q}$$

#### exact moments

where 
$$\begin{aligned} 
  \textsf{moments (exact): } & \\
 & \sum_{i=0}^{n} d_{i} p_{i} = \mu \\ 
 & \sum_{i=0}^{n} d_{i}^2 p_{i} = \mu^2 +\sigma^2, \forall i, \quad \textsf{can use sample mean/var}
 \end{aligned}$$

$$\begin{aligned}
  & p^\star = \frac{\beta\mathbf N}{h - \theta \mathbf 1 - \alpha d - w(d\bullet d)}
\end{aligned}$$



$$\begin{aligned}
  & \max_{x, ...} 
  \theta + \beta \gamma + \alpha \mu + w(\hat \mu^2 + \hat \sigma^2) +\beta N + t \\
  \textbf {s.t.} \\
  & (\mathbf q, \beta\mathbf N, t) \in \mathcal K_{\exp} \\
  & \beta \ge 0 \\
  & \mathbf q \equiv - \mathbf h - \theta \mathbf 1 - ...\ge 0\\
  & \mathbf q + \theta \mathbf 1 + ...+  b\cdot(d - x) \le 0\\
  & \mathbf q +  \theta \mathbf 1 + ... + h\cdot(x - d) \le 0\\
  & x \in D 
\end{aligned}$$

#### JuMP code

> Remark Julia MathOptInterface uses slight different notation on Cones,  refer to [MathOptInterface API](https://jump.dev/MathOptInterface.jl/v0.9.1/apireference/#MathOptInterface.ExponentialCone)

In [1]:
using JuMP
using Distributions
using StatsBase
using MosekTools
using Plots
using LinearAlgebra
import MathOptInterface
const MOI = MathOptInterface

plotly()

# truncation @[0-200]
int_trunc = x -> round(min(max(x, 0), 200))

# sample object
struct Sample
    h::Float64
    b::Float64
    N::Int32
    n::Int32
    S
    H
    mu::Float64
    sig::Float64
end

# solution object
struct Sol
    x::Float64
    model::JuMP.Model
    p::Array{Float64,1}
end 

h = b = 1
N, n = 1000, 200
S1 = int_trunc.(rand(Normal(50, 50), N))
H1 = fit(Histogram, S1, 0:n)
mu1, sig1 = mean_and_std(S1)
S2 = int_trunc.(rand(Exponential(50), N))
H2 = fit(Histogram, S2, 0:n)
mu2, sig2 = mean_and_std(S2)
d = Array(1:200)

sample1 = Sample(h,b,N,n,S1, H1, mu1, sig1)
sample2 = Sample(h,b,N,n,S2, H2, mu2, sig2)

# simple lambda function
x_scarf = s -> s.mu
x_ro_1 = x_scarf(sample1)
x_ro_2 = x_scarf(sample2)

48.497

In [2]:
# solution evaluation
mutable struct Eval
    sol::Sol
    sample::Sample
    d::Array
    
    # objectives
    obj_worse::Float64
    obj_true::Float64
    
    function Eval(sol::Sol, sample::Sample, d::Array)
        x = new(sol, sample, d, 0, 0)
        h = max.(
            (d .- sol.x) .* sample.b, 
            (sol.x .- d) .* sample.h
        )
        x.obj_true = sum(sample.H.weights .* h) / sample.N
        x.obj_worse = sum(sol.p .* h)
        x
    end
end

In [3]:
# 1. pure lro model
function lro_nv_model(sample)
    h, b, N, n = sample.h, sample.b, sample.N, sample.n
    H = sample.H.weights
    Hs = [i for i in H if i > 0]
    gamma = sum(Hs .* (log.(Hs./N))) - 1/2 * quantile.(Gamma(n-1), [0.95])[1]
    model = JuMP.Model()
    @variable(model, theta)
    @variable(model, beta >= 0)
    @variable(model, q[1:n] >= 0)
    @variable(model, x >= 0)

    @constraint(model, q .+ b * (d .- x) .+ theta .<= 0)
    @constraint(model, q .+ h * (x .- d) .+ theta .<= 0)

    @variable(model, t[1:n])
    @constraint(model, KL_DEV[i=1:n], [t[i], H[i] * beta, q[i]] in MOI.ExponentialCone())
    obj_expr = 
    begin
        theta + beta * (gamma + N) + dot(ones(n), t)
    end
    @objective(model, Max, obj_expr)
    set_optimizer(model, Mosek.Optimizer)
    optimize!(model)
    x_sol = value.(x)
    p_sol = value.(beta).*H ./ value.(q)
    return Sol(x_sol, model, p_sol)
end

lro_nv_model (generic function with 1 method)

In [4]:

lro_sol1 = lro_nv_model(sample1)
# lro_sol2 = lro_nv_model(sample2)
# plot sampling distribution and worse-case


lro_p1 = plot(1:n, [sample1.H.weights lro_sol1.p * N],
    label=reshape(["@true", "@worst-case"], 1, 2),
    title="normal"
)


Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1000            
  Cones                  : 200             
  Scalar variables       : 1003            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

<!DOCTYPE html>
 
 
 Plots.jl

In [5]:
# 2. lro + moments model
function lro_moment_nv_model(sample::Sample)
    h, b, N, n = sample.h, sample.b, sample.N, sample.n
    H, u, sig = sample.H.weights, sample.mu, sample.sig 
    Hs = [i for i in H if i > 0]
    gamma = sum(Hs .* (log.(Hs./N))) - 1/2 * quantile.(Gamma(n-1), [0.95])[1]
    model = JuMP.Model()
    @variable(model, theta)
    @variable(model, beta >= 0)
    @variable(model, q[1:n] >= 0)
    @variable(model, x >= 0)
    @variable(model, a)
    @variable(model, w)

    @constraint(model, q .+ b * (d .- x) .+ theta .+ (d .* a) .+ (d .* d .* w) .<= 0)
    @constraint(model, q .+ h * (x .- d) .+ theta .+ (d .* a) .+ (d .* d .* w) .<= 0)

    @variable(model, t[1:n])
    @constraint(model, KL_DEV[i=1:n], [t[i], H[i] * beta, q[i]] in MOI.ExponentialCone())
    obj_expr = 
    begin
        theta + a * u + w * (u^2 + sig^2) + beta * (gamma + N) + dot(ones(n), t)
    end
    @objective(model, Max, obj_expr)
    set_optimizer(model, Mosek.Optimizer)
    optimize!(model)
    x_sol = value.(x)
    p_sol = value.(beta) .* H ./ value.(q)
    return Sol(x_sol, model, p_sol)
end

lro_moment_nv_model (generic function with 1 method)

#### Wrap up results

In [6]:
samples = Dict(
    "normal" => sample1, 
    "exp" => sample2
)
models = Dict(
    "lro" => lro_nv_model,
    "lro_mm" => lro_moment_nv_model
)

Dict{String,Function} with 2 entries:
  "lro_mm" => lro_moment_nv_model
  "lro"    => lro_nv_model

In [7]:
data = []
results = Dict()
data = [(v = samples[k]; 
        eval = Eval(models[m](v), v, d); 
        results[k, m] = eval;
        a = [eval.sol.x eval.obj_true eval.obj_worse]) 
    for k in ["normal", "exp"] for m in ["lro", "lro_mm"]]
data = vcat(data...)

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1000            
  Cones                  : 200             
  Scalar variables       : 1003            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

4×3 Array{Float64,2}:
 59.8911  36.9653  42.6789
 53.9407  36.5232  40.4394
 47.0     31.793   49.9453
 32.0     30.645   38.0847

In [8]:
objective_value(results["exp", "lro_mm"].sol.model)

-38.08515039579801

In [11]:
using DataFrames

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1260


In [12]:
data

4×3 Array{Float64,2}:
 59.8911  36.9653  42.6789
 53.9407  36.5232  40.4394
 47.0     31.793   49.9453
 32.0     30.645   38.0847

In [17]:
df = DataFrame(
    Index=["normal, LRO", "normal, LRO_mm", "Exp, LRO", "Exp, LRO_mm"], 
    Sol=data[:, 1],
    True_obj=data[:, 2],
    Worst_obj=data[:, 3],
)

,Index,Sol,True_obj,Worst_obj
,String,Float64,Float64,Float64
1,"normal, LRO",59.8911,36.9653,42.6789
2,"normal, LRO_mm",53.9407,36.5232,40.4394
3,"Exp, LRO",47.0,31.793,49.9453
4,"Exp, LRO_mm",32.0,30.645,38.0847
